## Import

In [ ]:
import os
import numpy as np 
from astropy.io import fits 
import matplotlib.pyplot as plt 
import py_specrebin_vec
from matplotlib import rc
import pandas as pd
from scipy import optimize
path_name = '.'

## Variable Pannel

In [ ]:
mask_name = "A1M33P" #Enter your mask here!
grating = 600 #Keep the grating 600. BPT doesn't use 1200 grating.

## Define Wavelength

In [ ]:
new_wave_600 = np.arange(4000, 11000, .65) 
new_wave_1200 = np.arange(6000, 11000, .33) 

## Read Data

In [ ]:
Data = fits.open("./AGST Subtracted Spectra/{}_AGST_Subtracted_Spectra.fits.gz".format(mask_name))
AGST_RBFlux = Data[1].data #Read and extract data

## Function To Read ISM_EM_LINES.txt & Extract Slit # of Excluded Slits

In [ ]:
def get_exclusions():
    filepath = 'ISM_EM_LINES.txt'
    fp = open(filepath)
    all_data = []
    for line in (fp):
        mask_name = line.split(':')[0].split('_')[0]
        slit_number = line.split(':')[1].strip().split(" ")[0]
        if len(slit_number) == 2:
            slit_number = '0' + slit_number
        elif len(slit_number) == 1:
            slit_number = '00' + slit_number
        else:
            pass
        object_id = line.split(':')[1].strip().split()[1]
        data = {}
        data['mask_name'] = mask_name
        data['slit_number'] = slit_number
        data['object_id'] = object_id
        all_data.append(data)
    return all_data     

In [ ]:
def get_files_to_include(folder):
    import os
    list_of_files_to_include = []
    list_of_files_to_exclude = []
    serendip_files = []
    all_file_names_in_folder = os.listdir('data/{}'.format(folder))
    y = len(all_file_names_in_folder)
    print("The number of files in the folder is {0}".format(y))
    all_data = get_exclusions()
    len_all_data = len(all_data)
    for n in range(y):
        parts_of_file_name = all_file_names_in_folder[n].split(".")
        if parts_of_file_name[0] == 'spec1d': # avoids hidden DS_Store files on my mac
            object_id = parts_of_file_name[3]
            slit_number = parts_of_file_name[2]
            mask_name = parts_of_file_name[1]
            should_include = True
            should_exclude = True
            for k in range(len_all_data):
                if ((object_id == all_data[k]['object_id']) and (slit_number == all_data[k]['slit_number']) and (mask_name == all_data[k]['mask_name'])):
                    should_include = False
                    should_exclude = True
                if 'serendip' in object_id:
                    should_include = False
                    should_exclude = False
            if should_include == True:
                list_of_files_to_include.append(all_file_names_in_folder[n])       
            elif should_exclude == True:
                list_of_files_to_exclude.append(all_file_names_in_folder[n])
            elif should_include == False & should_exclude == False:
                serendip_files.append(all_file_names_in_folder[n])
    
    print('The number of files left after exclusions is {0}'.format(len(list_of_files_to_include)))
    
    return sorted(list_of_files_to_include), sorted(list_of_files_to_exclude), sorted(serendip_files)


In [ ]:
def get_slit_nums(files):
    
    slit_nums = []
    
    if len(files) > 1:
    
        for i in range(len(files)):
            parts_of_file_name = files[i].split(".")
            slit_num = parts_of_file_name[2]
            slit_nums.append(int(slit_num))
            
    return slit_nums

## Calls To Get Slit Numbers

In [ ]:
#filtering files
list_of_files_to_include, list_of_files_to_exclude, list_of_serendip_files = get_files_to_include(mask_name)

file_names = list_of_files_to_include
file_names_exclude = list_of_files_to_exclude
file_names_serendip = list_of_serendip_files
file_names_all = list_of_files_to_include + list_of_files_to_exclude

In [ ]:
slit_nums = get_slit_nums(file_names) #get slit # of INCLUDED slits
slit_nums_exclude = get_slit_nums(file_names_exclude) #get slit # of EXCLUDED slits
all_slit_nums = get_slit_nums(file_names_all) #slit # of INCLUDED & EXCLUDED slits

print("Slit # to INCLUDE in median calculation: {0}".format(slit_nums))
print("Slit # to EXCLUDE: {0}".format(slit_nums_exclude))

## View ONE

In [ ]:
slit_number = 4
plt.plot(new_wave_600,AGST_RBFlux[slit_nums_exclude.index(slit_number)])
#plt.ylim(-50,100)
plt.xlim(4800,5000)
#plt.xlim(6500,6700)

## View ALL

In [ ]:
def Emission_View(slit_nums_exclude,AGST_RebinnedFlux): #Function design to display Hb, OIII, Ha, and NII lines.
    for index in range(len(slit_nums_exclude)):
        fig,ax = plt.subplots(2,2) 
        
        #H Beta
        fig.suptitle("Mask:{} Slit #:{}".format(mask_name,slit_nums_exclude[index]),fontweight="bold")
        ax[0,0].plot(new_wave_600,AGST_RebinnedFlux[index])
        ax[0,0].set_xlim(4840,4880)
        ax[0,0].set_ylim(-10,100)
        ax[0,0].axvline(4861,-500,500,linestyle="--",c="black")
        ax[0,0].set_title("H_Beta")
        ax[0,0].set_xlabel("Wavelength ($\AA$)")
        ax[0,0].set_ylabel("Flux (e/hr)")
        
        #OIII
        ax[1,0].plot(new_wave_600,AGST_RebinnedFlux[index])
        ax[1,0].set_xlim(4980,5020)
        ax[1,0].set_ylim(-10,100)
        ax[1,0].axvline(5007,-500,500,linestyle="--",c="black")
        ax[1,0].set_title("OIII")
        ax[1,0].set_xlabel("Wavelength ($\AA$)")
        ax[1,0].set_ylabel("Flux (e/hr)")
        
        #H Alpha
        ax[0,1].plot(new_wave_600,AGST_RebinnedFlux[index])
        ax[0,1].set_xlim(6540,6580)
        ax[0,1].set_ylim(-10,100)
        ax[0,1].axvline(6562,-500,500,linestyle="--",c="black")
        ax[0,1].set_title("H_Alpha")
        ax[0,1].set_xlabel("Wavelength ($\AA$)")
        ax[0,1].set_ylabel("Flux (e/hr)")
        
        #NII
        ax[1,1].plot(new_wave_600,AGST_RebinnedFlux[index])
        ax[1,1].set_xlim(6565,6605)
        ax[1,1].set_ylim(-10,100)
        ax[1,1].axvline(6583,-500,500,linestyle="--",c="black")
        ax[1,1].set_title("NII")
        ax[1,1].set_xlabel("Wavelength ($\AA$)")
        ax[1,1].set_ylabel("Flux (e/hr)")
        plt.tight_layout()
        
        plt.show()
            

In [ ]:
Emission_View(slit_nums_exclude,AGST_RBFlux) #Call the function above

## Integral Testing Area

## Gaussian Integral Function

In [ ]:
def gaussian(x, amplitude, mean, stddev):
    return amplitude*np.exp(-((x - mean)**2/(2*(stddev**2))))

In [ ]:
def area(window_min,window_max,addons_window_pixels,flux,wave=new_wave_600):
    
    #1st window (Search window)
    WavWin_Bool = (wave > window_min) & (wave < window_max) #Boolean array of wavelength between two set limits
    Window_X = wave[WavWin_Bool] #Use boolean array to index x-values of first window
    Window_Y = flux[WavWin_Bool] #Use boolean array to index y-values of first window
    
    #2nd window (Capture window)
    Peak_Idx = list(flux).index(max(Window_Y)) #Determine the index of the peak
    New_Window_Lim = np.array([Peak_Idx-addons_window_pixels,Peak_Idx+addons_window_pixels]) #Set pixel length of second window
    Wavelength = wave[New_Window_Lim[0]:New_Window_Lim[1]] #Wavelength of second window
    Flux = flux[New_Window_Lim[0]:New_Window_Lim[1]] #Flux of second window
    
    #Calculate Amplitude, Mean, and Standard Dev
    p0_A = max(Flux) #Guess Amplitude
    p0_mu = Wavelength[list(Flux).index(p0_A)] #Guess Mean
    p0_sigma = np.std(Wavelength) #Guess Standard Dev
    popt,_ = optimize.curve_fit(gaussian,Wavelength,Flux,p0=[p0_A,p0_mu,p0_sigma])
    
    #Calculate line strength
    Amplitude = popt[0]
    STD = popt[2]
    Line_STR = Amplitude * STD #Using the Amp*SD/0.3989 formula. Ratio ignore constant.
    
    #Visual inspection
    gaussian_x = np.linspace(wave[New_Window_Lim[0]],wave[New_Window_Lim[1]],100)
    gaussian_y = gaussian(gaussian_x,popt[0],popt[1],popt[2])
    plt.plot(Window_X,Window_Y,color="black",label="First Window")
    plt.plot(Wavelength,Flux,color="blue",label="Second Window")
    plt.plot(gaussian_x,gaussian_y,color="red",linestyle="--",label="Gaussian")
    plt.vlines(wave[New_Window_Lim[0]],-500,500,linestyle="--",color="green",label="2nd Window Limit")
    plt.vlines(wave[New_Window_Lim[1]],-500,500,linestyle="--",color="green",label="2nd Window Limit")
    plt.xlim(window_min,window_max)
    plt.ylim(-10,150)
    plt.legend()
    
    return Line_STR

In [ ]:
Line_STR = area(4840,4900,15,data_of_s49)

### Gaussian Fitting 

In [ ]:
from scipy import optimize

def gaussian(x, amplitude, mean, stddev):
    return amplitude*np.exp(-((x - mean)**2/(2*(stddev**2))))

In [ ]:
x_original = new_wave_600[1310:1340]
#x_original = new_wave_600[3960:3990]
y_original = data_of_s49[1310:1340]
#y_original = data_of_s49[3960:3990]
popt, _ = optimize.curve_fit(gaussian,x_original,y_original,p0=[max(y_original),x_original[list(y_original).index(max(y_original))],np.std(x_original)])


In [ ]:
popt

In [ ]:
plt.plot(x_original, y_original,label="Original Data")
x_gaussian = np.linspace(min(x_original),max(x_original),200)
y_gaussian = gaussian(x_gaussian, *popt)
plt.plot(x_gaussian, gaussian(x_gaussian, *popt),label="Gaussian Fit")
#plt.hlines(max(y_gaussian)/2,min(x_gaussian),max(x_gaussian),color='black',linestyle="--",label="50% Threshold")
plt.legend()

In [ ]:
import cmath
def quad_formula(a,b,c):
    return [(-b+cmath.sqrt((b**2)-(4*a*c)))/(2*a), 
            (-b-cmath.sqrt((b**2)-(4*a*c)))/(2*a)]

In [ ]:
amp = popt[0]
mu = popt[1]
sigma = popt[2]
a_quad = 1
b_quad = -2*mu
c_quad = mu**2 + np.log((max(y_gaussian)/2)/amp)*2*(sigma**2)

In [ ]:
intercepts = quad_formula(a_quad,b_quad,c_quad)

In [ ]:
intercepts

In [ ]:
from scipy import integrate 
area_test = integrate.quad(gaussian,intercepts[1],intercepts[0],args=(popt[0],popt[1],popt[2]))

In [ ]:
tot_area_gauss = area_test - ((max(gaussian(x_gaussian, *popt))/2)*(intercepts[0]-intercepts[1]))

In [ ]:
print("Total Area (Gaussian Method): {}".format(tot_area_gauss[0]))

In [ ]:
plt.plot(x_original, y_original,label="Original Data")
x_gaussian = np.linspace(min(x_original),max(x_original),200)
y_gaussian = gaussian(x_gaussian, *popt)
plt.plot(x_gaussian, gaussian(x_gaussian, *popt),label="Gaussian Fit")
plt.hlines(max(y_gaussian)/2,min(x_gaussian),max(x_gaussian),color='black',linestyle="--",label="50% Threshold")
plt.legend()

### Reimann Sum Integral Test

In [ ]:
index_of_s49 = slit_nums_exclude.index(49)
data_of_s49 = AGST_RBFlux[index_of_s49]

#H Beta
fig,ax=plt.subplots(2,2)
fig.suptitle("Slit #49 Testting",fontweight="bold")
ax[0,0].plot(new_wave_600,data_of_s49)
ax[0,0].set_xlim(4840,4880)
ax[0,0].set_ylim(-10,100)
ax[0,0].axvline(4861,-500,500,linestyle="--",c="black")
ax[0,0].axvline(new_wave_600[1310],-500,500,linestyle="--",c="red")
ax[0,0].axvline(new_wave_600[1340],-500,500,linestyle="--",c="red")
ax[0,0].set_title("H_Beta")
ax[0,0].set_xlabel("Wavelength")
ax[0,0].set_ylabel("Flux")

#OIII
ax[1,0].plot(new_wave_600,data_of_s49)
ax[1,0].set_xlim(4980,5020)
ax[1,0].set_ylim(-10,100)
ax[1,0].axvline(5007,-500,500,linestyle="--",c="black")
ax[1,0].axvline(new_wave_600[1534],-500,500,linestyle="--",c="red")
ax[1,0].axvline(new_wave_600[1564],-500,500,linestyle="--",c="red")
ax[1,0].set_title("OIII")
ax[1,0].set_xlabel("Wavelength")
ax[1,0].set_ylabel("Flux")

#H Alpha
ax[0,1].plot(new_wave_600,data_of_s49)
ax[0,1].set_xlim(6540,6580)
ax[0,1].set_ylim(-10,100)
ax[0,1].axvline(6562,-500,500,linestyle="--",c="black")
ax[0,1].axvline(new_wave_600[3928],-500,500,linestyle="--",c="red")
ax[0,1].axvline(new_wave_600[3958],-500,500,linestyle="--",c="red")
ax[0,1].set_title("H_Alpha")
ax[0,1].set_xlabel("Wavelength")
ax[0,1].set_ylabel("Flux")

#NII
ax[1,1].plot(new_wave_600,data_of_s49)
ax[1,1].set_xlim(6565,6605)
ax[1,1].set_ylim(-10,100)
ax[1,1].axvline(6583,-500,500,linestyle="--",c="black")
ax[1,1].axvline(new_wave_600[3960],-500,500,linestyle="--",c="red")
ax[1,1].axvline(new_wave_600[3990],-500,500,linestyle="--",c="red")
ax[1,1].set_title("NII")
ax[1,1].set_xlabel("Wavelength")
ax[1,1].set_ylabel("Flux")
plt.tight_layout()

Set window of 30 pixels

In [ ]:
halfmax_of_s49 = max(data_of_s49[1310:1340])/2 #find 1/2 of the maximum value 
above_halfmax = data_of_s49[1310:1340] > halfmax_of_s49 #boolean of all values above determined halfmax

In [ ]:
plt.plot(new_wave_600,data_of_s49)
plt.scatter(new_wave_600[1310:1340][above_halfmax],data_of_s49[1310:1340][above_halfmax],c="black")
plt.hlines(halfmax_of_s49,0,10000,linestyle="--",color="black")
plt.title("Mark Datapoints Above Threshold")
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.xlim(4840,4880)
plt.ylim(-10,100)

In [ ]:
above_halfmax_index = np.where(above_halfmax == True) #find index of all True boolean from above_halfmax
left_ends_idx = above_halfmax_index[0][0] - 1 #append first left point below halfmax to current index list
right_ends_idx = above_halfmax_index[0][-1] + 1 #append first right point below halfmax to current index list
above_halfmax_index = np.sort(np.append(above_halfmax_index,[left_ends_idx,right_ends_idx]))

In [ ]:
Marked_X_Vals = new_wave_600[1310:1340][above_halfmax_index]
Marked_Y_Vals = data_of_s49[1310:1340][above_halfmax_index]

In [ ]:
plt.plot(new_wave_600,data_of_s49)
plt.scatter(Marked_X_Vals,Marked_Y_Vals,c="black")
plt.hlines(halfmax_of_s49,0,10000,linestyle="--",color="black")
plt.title("Above Threshold Modified")
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.xlim(4855,4865)
plt.ylim(-10,100)

In [ ]:
Linear_Func = lambda x,m,b: (m*x)+b
Y_Intercept_Func = lambda x,y,m: y-(m*x)
X_Value_Func = lambda y,m,b: (y-b)/m

In [ ]:
m = np.diff(Marked_Y_Vals)/np.diff(Marked_X_Vals)
b = Y_Intercept_Func(Marked_X_Vals[1:],Marked_Y_Vals[1:],m)
                     

In [ ]:
Left_End_x = X_Value_Func(halfmax_of_s49,m[0],b[0])
Right_End_x = X_Value_Func(halfmax_of_s49,m[-1],b[-1])

In [ ]:
plt.plot(new_wave_600,data_of_s49)
plt.scatter(Marked_X_Vals,Marked_Y_Vals,c="black")
plt.scatter(Left_End_x,halfmax_of_s49,c="red")
plt.scatter(Right_End_x,halfmax_of_s49,c="red")
plt.hlines(halfmax_of_s49,0,10000,linestyle="--",color="black")
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.xlim(4855,4865)
plt.ylim(-10,100)

In [ ]:
Marked_X_Vals

In [ ]:
Marked_X_Vals[0] = Left_End_x

In [ ]:
Marked_X_Vals

In [ ]:
Marked_X_Vals[-1] = Right_End_x

In [ ]:
Marked_X_Vals

In [ ]:
Marked_Y_Vals[0] = halfmax_of_s49
Marked_Y_Vals[-1] = halfmax_of_s49

In [ ]:
Marked_Y_Vals

In [ ]:
plt.plot(new_wave_600,data_of_s49)
plt.scatter(Marked_X_Vals,Marked_Y_Vals,c="black")
plt.hlines(halfmax_of_s49,0,10000,linestyle="--",color="black")
plt.title("Final Datapoints Plotted")
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.xlim(4855,4865)
plt.ylim(-10,100)
plt.fill_between(Marked_X_Vals,Marked_Y_Vals,halfmax_of_s49)

In [ ]:
from scipy import integrate
tot_area = 0
for idx in range(len(m)):
    area = integrate.quad(Linear_Func,Marked_X_Vals[idx],Marked_X_Vals[idx+1],args=(m[idx],b[idx])) 
    tot_area += (area[0]-((Marked_X_Vals[idx+1]-Marked_X_Vals[idx])*halfmax_of_s49))
    print(tot_area)

In [ ]:
print("Total Area (Reimann Sum Method): {}".format(tot_area))